# Training Yolo v5 using custom data annotation

After successful data prepration, we are trying to detect the Hexbugs using `Yolo v5`. We are doing this using the `Pytorch` library. The original data annotations that we were provided with, contained only the `(x, y)` position of the Hexbugs' heads' position. To train a `Yolo` model, we need to do extra data annotations to obtain the bouding boxes. To do so, we used [Roboflow](https://roboflow.com/). Roboflow is an awesome platform for data annotation which allows teams to work on uploaded data in groups. After annotating the frames, we exported the dataset as a `Yolo v5 dataset`. Please mention that the original libraries are implemented in `Pytorch`. Therefore, make sure that you have access to proper GPUs to run the code.

### Check Nvidia driver

In [2]:
!nvidia-smi

Wed May 10 13:23:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   41C    P8    14W /  N/A |     54MiB /  6144MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Required imports

* `os` is required to manage directories.
* `ultralytics` is essential to load `Yolo`
* `roboflow` is required to download the dataset from the server.

In [4]:
# Pip install method (recommended)
!pip install ultralytics==8.0.20
!pip install roboflow --quiet

import os
import glob

from credentials import YOUR_API_KEY
from IPython import display
from IPython.display import display, Image, clear_output
import ultralytics
from ultralytics import YOLO
from roboflow import Roboflow

ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.9.16 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5947MiB)
Setup complete ✅ (20 CPUs, 15.4 GB RAM, 140.7/544.6 GB disk)


In [5]:
# Setting Home variable to work easier with the file paths later.
HOME = os.getcwd()
print(HOME)

/home/farzam/workspace/python/ml/Courses/TRACO


### Resetting environment

In [7]:
# Removing already existing `datasets` directory
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/home/farzam/workspace/python/ml/Courses/TRACO/datasets


### Downloading dataset from `Roboflow`

In [9]:
# Using `Roboflow` library to access and downloading the dataset.
rf = Roboflow(api_key=YOUR_API_KEY)
project = rf.workspace("hexbugs").project("bugbusters")
dataset = project.version(5).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to BugBusters-5 in yolov5pytorch:: 100%|█| 2140/2


In [ ]:
%cd {HOME}
# using the already implemented `yolo` package to perform a training task.
# It'll download the `Yolov8s.pt` mdoel for reference and use its already trained weights for transfer learning.
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=100 plots=True

/home/farzam/workspace/python/ml/Courses/TRACO
Ultralytics YOLOv8.0.20 🚀 Python-3.9.16 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5947MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/home/farzam/workspace/python/ml/Courses/TRACO/datasets/BugBusters-5/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      6/100      4.88G      1.046     0.6617      1.066          8        640: 100%|██████████| 47/47 [00:10<00:00,  4.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.07it/s]
                   all        209        209      0.603      0.943      0.738      0.544
                HexBug        209        209      0.603      0.943      0.738      0.544

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      7/100      4.88G       1.05     0.6405      1.079          4        640: 100%|██████████| 47/47 [00:10<00:00,  4.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.26it/s]
                   all        209        209      0.941      0.918      0.977      0.645
                HexBug        209        209     


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     22/100      4.88G     0.9108     0.5109     0.9972          8        640: 100%|██████████| 47/47 [00:10<00:00,  4.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.00it/s]
                   all        209        209      0.999          1      0.995      0.806
                HexBug        209        209      0.999          1      0.995      0.806

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     23/100      4.88G     0.9356     0.5193      1.027         10        640: 100%|██████████| 47/47 [00:10<00:00,  4.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.23it/s]
                   all        209        209          1          1      0.995      0.815
                HexBug        209        209     


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     38/100      4.88G     0.8604     0.4758      0.988         10        640: 100%|██████████| 47/47 [00:10<00:00,  4.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.13it/s]
                   all        209        209      0.999          1      0.995      0.817
                HexBug        209        209      0.999          1      0.995      0.817

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     39/100      4.88G      0.856     0.4785      0.993         11        640: 100%|██████████| 47/47 [00:10<00:00,  4.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.20it/s]
                   all        209        209          1          1      0.995      0.822
                HexBug        209        209     


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     54/100      4.88G     0.7668     0.4271     0.9577         11        640: 100%|██████████| 47/47 [00:10<00:00,  4.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.22it/s]
                   all        209        209          1          1      0.995      0.825
                HexBug        209        209          1          1      0.995      0.825

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     55/100      4.88G     0.7946     0.4354     0.9628          5        640: 100%|██████████| 47/47 [00:10<00:00,  4.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.22it/s]
                   all        209        209          1          1      0.995       0.83
                HexBug        209        209     


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     70/100      4.88G     0.7219     0.3849     0.9359          9        640: 100%|██████████| 47/47 [00:10<00:00,  4.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.18it/s]
                   all        209        209          1          1      0.995      0.832
                HexBug        209        209          1          1      0.995      0.832

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     71/100      4.88G     0.7192     0.3664     0.9346          8        640: 100%|██████████| 47/47 [00:09<00:00,  4.74it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.09it/s]
                   all        209        209          1          1      0.995      0.835
                HexBug        209        209     


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     86/100      4.88G     0.6587     0.3234     0.9183         10        640: 100%|██████████| 47/47 [00:10<00:00,  4.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.25it/s]
                   all        209        209          1          1      0.995       0.85
                HexBug        209        209          1          1      0.995       0.85

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     87/100      4.88G     0.6628     0.3392     0.9248          4        640: 100%|██████████| 47/47 [00:10<00:00,  4.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.14it/s]
                   all        209        209          1          1      0.995      0.849
                HexBug        209        209     

In [ ]:
!ls {HOME}/runs/detect/train/

### Results Visualization
Most of the times, it's a great idea to create a confusion matrix to understand the performance of the model. Here, we plot the confusion matrix, but it only has one class, `HexBug`.

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

### Plotting metrics
We plot `loss`, `recall`, and `mAP` as metrics to analyze the model performance better.

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

### Plotting results over validation set

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

### Performing validation over the validation set

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

### Performing detection over test set

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

### Plotting the detection result over unseen dataset

In [ ]:
for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

### Result

Now we have a fully trained `Yolo` model that performs quite good in detecting Hexbugs.
Now it's time to use this model to detect Hexbugs in the frames and get rid of void areas.
The best model is now located at `runs/detect/train/weights/best.pt`.